# Test notebook for Sx operations.

In [1]:
import asyncio
from json import dumps, loads
import queue
import requests
from minio import Minio
import tempfile
import pyarrow.parquet as pq
import pytest
from config import running_backend, default_container
from servicex_test_utils import wait_for_request_done, get_servicex_request_data
import servicex
from func_adl import EventDataset
from func_adl_xAOD import use_exe_servicex
import uproot_methods
import numpy
import csv
from numpy import genfromtxt

#Datasets:

localds://mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00

user.emmat:user.emmat.mc16_13TeV.311311.MadGraphPythia8EvtGen_A14NNPDF31LO_HSS_LLP_mH125_mS15.mc16d.200131.forsX_trees.root

In [20]:
def test_func_adl_simple_jet_pts():
    query = EventDataset('localds://mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00') \
    .SelectMany('lambda e: (e.Jets("AntiKt4EMTopoJets"))') \
    .Where('lambda j: (j.pt())>30000') \
    .Select('lambda j: (j.pt())') \
    .AsPandasDF("JetPt") \
    .value(executor = lambda a: use_exe_servicex(a, endpoint='http://localhost:5000/servicex'))
    
    return query

In [2]:
def test_func_adl_uproot_simple_jet_pts():
    query = EventDataset('user.emmat:user.emmat.mc16_13TeV.311311.MadGraphPythia8EvtGen_A14NNPDF31LO_HSS_LLP_mH125_mS15.mc16d.200131.forsX_trees.root') \
    .SelectMany('lambda e: (e.Jets("AntiKt4EMTopoJets"))') \
    .Where('lambda j: (j.pt())>30000') \
    .Select('lambda j: (j.pt())') \
    .AsROOTTTree("data.root", "analysis", "JetPt") \
    .value(executor = lambda a: use_exe_servicex(a, endpoint='http://localhost:5000/servicex'))
    
    return query

In [6]:
data = test_func_adl_uproot_simple_jet_pts()

FuncADLServerException: Unable to use ServiceX to fetch a result in the form ResultTTree

In [5]:
def test_division():
	query = EventDataset('localds://mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00') \
	.SelectMany('lambda e: (e.Jets("AntiKt4EMTopoJets"))') \
	.Where('lambda j: (j.pt())>30000') \
	.Select('lambda j: (j.pt())') \
	.AsPandasDF("JetPt") \
	.value(executor = lambda a: use_exe_servicex(a, endpoint='http://localhost:5000/servicex'))
    
data_divided = test_division()/1000

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'